<a href="https://colab.research.google.com/github/kostkalab/mol-evol_notebooks/blob/main/lecture_mol_evol_03_13_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Transition matrix
Here we look at a sequence of nucleotides (say `ACGTACTTA`) describe "transitions" to an alternative sequence (say `ACGTACAAA`). In our approach we will consider each nucleotide independently of its neighbors. Then the question becomes for each nucleotide, what is the probabillity to "transition" into any of the three other nucleotides (or stay the same). This can be quantified by a $4\times4$ **transition matrix** generally denoted by $P$.

$P_{ij} = \text{Pr[nucleotide i} \rightarrow \text{nucleotide j]}$

As $P$ denotes transition probabilites as explained above, we already know:
* Its entries are non-negative, i.e., $P_{ij} \geq 0$.
* Its rows sum to one, i.e., $\sum_{j=1}^4 P_{ij} = 1$.
* Its diagnoal entries are the probability of a nucleotide not changing.

In the following we'll explore how to work with such a transition matrix in plain `R`.

***First we make a simple transition matrix***

In [ ]:
#- Define four nucleotides
NUCLEOTIDES = c('A','C','G','T')

#- Make a transition matrix
P = matrix(1/4, ncol=4, nrow=4)
colnames(P) <- rownames(P) <- NUCLEOTIDES
P

,A,C,G,T
A,0.25,0.25,0.25,0.25
C,0.25,0.25,0.25,0.25
G,0.25,0.25,0.25,0.25
T,0.25,0.25,0.25,0.25


***Next a sequence of nucleotides***

In [ ]:
#- A random sequence of nucleotides
set.seed(16386489)
seq1 <- sample(NUCLEOTIDES, 40, replace = TRUE)
seq1

[1] "T" "C" "A" "C" "T" "G" "C" "A" "T" "G" "T" "G" "A" "C" "C" "A" "G" "G" "C"
[20] "G" "T" "C" "G" "G" "G" "C" "T" "C" "G" "C" "T" "C" "G" "C" "A" "T" "T" "C"
[39] "T" "A"

***Nucleotide substitutions***

In [ ]:
#- Change our current subsitutions matrix
diag(P) <- 1.5
P       <-  P/rowSums(P)
P

,A,C,G,T
A,0.6666667,0.1111111,0.1111111,0.1111111
C,0.1111111,0.6666667,0.1111111,0.1111111
G,0.1111111,0.1111111,0.6666667,0.1111111
T,0.1111111,0.1111111,0.1111111,0.6666667


In [ ]:
#- create an empty sequence
seq1_a <- rep(NA, length(seq1))

#- "evolve" seq1 and store the result in seq1_a
for(i in seq_len(seq1 |> length())){
  nuc       <- seq1[i]
  new_nuc   <- sample(NUCLEOTIDES, size = 1, prob = P[nuc,])
  seq1_a[i] <- new_nuc
}

#- NOTE: the "R-way" of doing this would be
#  seq1_a <- sapply(seq1, \(nuc) sample(NUCLEOTIDES, size = 1, prob = P[nuc,]))

#- we can now, e.g., count differences
is_different <- seq1 != seq1_a
is_different |> table()

#- QUESTION: what would you expect? Maybe 13.3?

is_different
FALSE  TRUE 
   24    16 

In [ ]:
#- we can also print/visualize the new sequence we created
paste0( paste0(seq1, collapse=""),
        '\n',
        paste0(c("|"," ")[is_different + 1], collapse=""),
        '\n',
        paste0(seq1_a, collapse=""),
        collapse="") |> cat()       

TCACTGCATGTGACCAGGCGTCGGGCTCGCTCGCATTCTA
 ||  |||  | |||  | | ||||||  |||||  |  |
ACAGGGCAGCTAACCGCGTGACGGGCTGTCTCGCCGTGGA

In [ ]:
#- and since we might want to do this again, we'll write a function for is
print_seqs <- function(seq1, seq2){
#----------------------------------
  
  diffs <- c(' ', '|')[(seq1 != seq2) + 1]
  line1 <- paste0(seq1,  collapse = "")
  line2 <- paste0(diffs, collapse = "")
  line3 <- paste0(seq2,  collapse = "")
  
  paste0(line1,'\n',line2,'\n', line3, '\n', collapse ='') |> cat()
  
}

print_seqs(seq1, seq1_a)

TCACTGCATGTGACCAGGCGTCGGGCTCGCTCGCATTCTA
|  ||   || |   || | |      ||     || || 
ACAGGGCAGCTAACCGCGTGACGGGCTGTCTCGCCGTGGA


***Diagonal elements and (expected) number of substitutions***

Here we choose a parameter $\delta \in [0,1]$ and set $P_{ii} = \delta$ and $P_{ij} = (1-\delta)/3$. 

- What are the expected number of substitutions in a sequence of length $l$?
- What happens if $\delta = 1$?
- What when $\delta \in (0.25, 1)$?
- What when $\delta = 0.25$?
- What when $\delta \in (0, 0.25)$?
- What when $\delta = 0$?

# Multi-step transitions

In the current framework we can ask, what is the probability of going from nucleotide `i` to nucleotide `j` in two (or more) "steps". One step is one "application" of the transition matrix described above.

Not that for the **matrix product** of the transition matrix wiht itself we have:

$PP = \{PP\}_{ij}$

$(PP)_{ij} = \sum_{k=1}^4 P_{ik}P_{kj}$

Which is the sum over all possitilities `k` for nucleotide `i` transitioning into another nucleotide `k` and then `k` transitioning into `j`. 

* Two step transition: $PP = P^2$
* Three step transition: $PPP = P^3$
* Four step transition: $PPPP = P^4$
* ...

In [ ]:
P

,A,C,G,T
A,0.6666667,0.1111111,0.1111111,0.1111111
C,0.1111111,0.6666667,0.1111111,0.1111111
G,0.1111111,0.1111111,0.6666667,0.1111111
T,0.1111111,0.1111111,0.1111111,0.6666667


In [ ]:
P %*% P

,A,C,G,T
A,0.4814815,0.1728395,0.1728395,0.1728395
C,0.1728395,0.4814815,0.1728395,0.1728395
G,0.1728395,0.1728395,0.4814815,0.1728395
T,0.1728395,0.1728395,0.1728395,0.4814815


In [ ]:
P %*% P %*% P

,A,C,G,T
A,0.3786008,0.2071331,0.2071331,0.2071331
C,0.2071331,0.3786008,0.2071331,0.2071331
G,0.2071331,0.2071331,0.3786008,0.2071331
T,0.2071331,0.2071331,0.2071331,0.3786008


In [ ]:
P %*% P %*% P %*% P

,A,C,G,T
A,0.3214449,0.2261850,0.2261850,0.2261850
C,0.2261850,0.3214449,0.2261850,0.2261850
G,0.2261850,0.2261850,0.3214449,0.2261850
T,0.2261850,0.2261850,0.2261850,0.3214449


In [ ]:
P %*% P %*% P %*% P %*% P %*% P %*% P 

,A,C,G,T
A,0.2622505,0.2459165,0.2459165,0.2459165
C,0.2459165,0.2622505,0.2459165,0.2459165
G,0.2459165,0.2459165,0.2622505,0.2459165
T,0.2459165,0.2459165,0.2459165,0.2622505


What are your observations?

Do you know what an eigenvector is?

In [ ]:
eigen(P)

eigen() decomposition
$values
[1] 1.0000000 0.5555556 0.5555556 0.5555556

$vectors
     [,1]        [,2]          [,3]       [,4]
[1,] -0.5  0.73526963  0.000000e+00  0.4575790
[2,] -0.5 -0.67649953  8.756053e-17  0.5406925
[3,] -0.5 -0.02938505 -7.071068e-01 -0.4991358
[4,] -0.5 -0.02938505  7.071068e-01 -0.4991358
